In [55]:
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
train_data = pd.read_csv('train.csv')
train_data.drop(['id','keyword','location'],axis=1,inplace=True)

In [57]:
train_data = train_data.replace('\n','',regex=True)

In [58]:
train_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [59]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [60]:
from nltk.corpus import stopwords
import re

def noiseRemoval(sentences):
    for i in range(len(sentences)):
        sentences[i] = sentences[i].lower()
        sentences[i] = re.sub(r'\W',' ',sentences[i]) 
        sentences[i] = re.sub(r'\d',' ',sentences[i]) 
        sentences[i] = re.sub(r'\s+',' ',sentences[i]) 
        words = nltk.word_tokenize(sentences[i])
        new=[]
        for word in words:
              if word not in stopwords.words('english'):
                new.append(word)
        sentences[i] = ' '.join(new)

    return sentences

In [61]:
for x in range(len(train_data.text)):
    sentences = sent_tokenize(train_data.text[x])
    train_data.text[x] = noiseRemoval(sentences)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
train_data.head()

,text,target
0,[deeds reason earthquake may allah forgive us],1
1,"[forest fire near la ronge sask, canada]",1
2,[residents asked shelter place notified office...,1
3,[people receive wildfires evacuation orders ca...,1
4,[got sent photo ruby alaska smoke wildfires po...,1


In [63]:
from nltk.stem.wordnet import WordNetLemmatizer

lem = WordNetLemmatizer()
def lemma(sentences):
    for i in range(len(sentences)):
        words = nltk.word_tokenize(sentences[i])
        words = [lem.lemmatize(word, pos='v') for word in words]
        sentences[i] = ' '.join(words)
    return sentences

In [64]:
for x in range(len(train_data.text)):
  train_data.text[x] = " ".join(lemma(train_data.text[x]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
train_data.head()

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents ask shelter place notify officer eva...,1
3,people receive wildfires evacuation order cali...,1
4,get send photo ruby alaska smoke wildfires pou...,1


In [66]:
X = train_data.text.values
y = train_data.target.values

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

In [68]:
model = make_pipeline(CountVectorizer() , MultinomialNB())
model.fit(X,y)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [69]:
model.score(X,y)

0.9164586890844608

In [70]:
test_data = pd.read_csv('test.csv')
test_data.drop(['keyword','location'],axis=1,inplace=True)
test_data.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [71]:
result = model.predict(test_data.text)

In [72]:
test_data['target'] = result

In [73]:
test_data.drop('text',axis=1,inplace=True)
test_data.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [77]:
test_data.to_csv('submission.csv',index=False)